In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sbn
from sklearn.neural_network import MLPClassifier
from utils import DataUtils
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
from utils import DataUtils
from CS7140_balancers import BinaryBalancer
from sklearn.metrics import confusion_matrix

## Load data and train the MLP classifier

In [ ]:
df_train, df_test = DataUtils.load_adults(use_torch_dataset=False, test=0.25)


hidden_layer_sizes = (128)
model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, 
                      learning_rate_init=.003, 
                      verbose=True, 
                      learning_rate="adaptive", 
                      max_iter=300)


trained_model = model.fit(X=df_train.loc[:, df_train.columns != "output"], 
                          y=df_train["output"])

### Choose Sensitive Attribute and Report Overall Accuracies

In [ ]:
categories = ["native-country", "race", "sex", "workclass"]
targets = ["Mexico", "Black", "Female", "Private"]

Y_accs = []
Y_adj_accs = []
for attribute in zip(categories, targets):
    
    category = attribute[0]
    sensitive_attribute = attribute[1]
    
    
    
    frequency = sum(df_train[category + "_ " + sensitive_attribute])/len(df_train)
    print(f"People with {category}={sensitive_attribute} account for {frequency*100:.4}% of the training data")
    a = np.array(df_test[category + "_ " + sensitive_attribute])

    y_pred =  trained_model.predict(df_test.loc[:, df_train.columns != "output"])
    y_true = np.array(df_test["output"])
    acc = sum(y_pred == y_true)/len(y_pred)
    print(f"The model's overall accuracy is {acc*100:.4}%")
    Y_accs.append(acc)
    
    # Fit balancer to adjust the predictions
    pb = BinaryBalancer(y=y_true, y_=y_pred, a=a, summary=False)
    pb.adjust(goal='odds', summary=False)
    fair_acc = sum(pb.get_adjusted_predictions() == y_true)/len(y_true)
    print(f"The model's overall accuracy after adjustment is {fair_acc*100:.4}%\n")
    Y_adj_accs.append(fair_acc)

In [ ]:
x = np.arange(len(categories) + 1)
width = 0.4
label_font = {
    'color':  'k',
    'weight': 'heavy',
    'size': 10
}

title_font = {
    'color':  'k',
    'size': 14
}

attributes = ["No \nAdjustment"] + [categories[i] + "=\n" + targets[i] for i in range(len(categories))]
plt.figure(figsize=(7,5), dpi=100)
plt.title("Impact of Fairness Adjustment on Utility", fontdict=title_font)
plt.bar(x, [Y_accs[0]] + Y_adj_accs, color=["darkgrey", "dodgerblue", "coral", "khaki", "mediumseagreen"])

plt.xticks(x, attributes)
plt.xlabel("Sensitive Group Adjusted For", fontdict=label_font)
plt.ylabel("Overall Prediction Accuracy", fontdict=label_font)


plt.ylim(0,1)
ax = plt.gca()
ax.autoscale(False)
ax.plot(plt.gca().get_xlim(), [Y_accs[0], Y_accs[0]], c="r", linestyle="--")
plt.draw()

#plt.savefig("Figures/overall_accuracies.jpg")

### Choose Sensitive Attribute and Report Per-Subgroup Accuracies

In [ ]:
Y_accs_on_group = []
Y_adj_accs_on_group = []
proportion_of_dataset = []
for attribute in zip(categories, targets):
    
    category = attribute[0]
    sensitive_attribute = attribute[1]
    frequency = sum(df_train[category + "_ " + sensitive_attribute])/len(df_train)
    proportion_of_dataset.append(frequency)
    print(f"People with {category}={sensitive_attribute} account for {frequency*100:.4}% of the training data")
    a = np.array(df_test[category + "_ " + sensitive_attribute])

    y_pred =  trained_model.predict(df_test.loc[:, df_train.columns != "output"])
    y_true = np.array(df_test["output"])
    
    a_bool = a.astype(bool)

    group_acc = sum(y_pred[a_bool] == y_true[a_bool])/len(y_pred[a_bool])
    print(f"The model's loss on this group (0) is {1 - group_acc:.4}")
    Y_accs_on_group.append(group_acc)
    # Fit balancer to adjust the predictions
#     print("Before Adjustment\n", confusion_matrix(y_true[a_bool], y_pred[a_bool]), "\n-----")
    pb = BinaryBalancer(y=y_true, y_=y_pred, a=a, summary=False)
    pb.adjust(goal='odds', summary=False)
    y_adj = pb.get_adjusted_predictions()
    fair_group_acc = sum(y_adj[a_bool] == y_true[a_bool])/len(y_true[a_bool])
    Y_adj_accs_on_group.append(fair_group_acc)
#     print("After Adjustment\n", confusion_matrix(y_true[a_bool], y_adj[a_bool]), "\n-------")
    print(f"The model's loss on this group (0) after adjustment is {1 - fair_group_acc:.4}\n")

In [ ]:

x = np.arange(len(categories))
width = 0.4
label_font = {
    'color':  'k',
    'weight': 'heavy',
    'size': 10
}

title_font = {
    'color':  'k',
    'size': 14
}

attributes =[categories[i] + "=\n" + targets[i] for i in range(len(categories))]
plt.figure(figsize=(7,5), dpi=100)
plt.title("Impact of Fairness Adjustment on Utility", fontdict=title_font)
plt.bar(x - width,  Y_accs_on_group, width = width, color="dodgerblue")
plt.bar(x, Y_adj_accs_on_group, width = width, color="coral")

plt.xticks(x - width/2, attributes)
plt.xlabel("Sensitive Group Adjusted For", fontdict=label_font)
plt.ylabel("Prediction Accuracy on Sensitive Group", fontdict=label_font)
plt.legend(["Original Model ($\hat{Y}$)", "Adjusted Model ($Y^{*}$)"], loc="lower right")
plt.savefig("Figures/subgroup_acuracy.jpg")

## Report Calibration Statistics before and after adjustment

In [ ]:
C_Y0 = []
C_Yhat0 = []
C_Ystar0 = []

C_Y1 = []
C_Yhat1 = []
C_Ystar1 = []

for attribute in zip(categories, targets):
    category = attribute[0]
    sensitive_attribute = attribute[1]

    frequency = sum(df_train[category + "_ " + sensitive_attribute])/len(df_train)
    print(f"People with {category}={sensitive_attribute} account for {frequency*100:.4}% of the training data")
    a = np.array(df_test[category + "_ " + sensitive_attribute])

    y_pred =  trained_model.predict(df_test.loc[:, df_train.columns != "output"])
    y_true = np.array(df_test["output"])

    # Fit balancer to adjust the predictions
    pb = BinaryBalancer(y=y_true, y_=y_pred, a=a, summary=False)
    pb.adjust(goal='odds', summary=False)

    calibration_table, _ = pb.summary(return_stats=True)
    
    # 0 is not the sensitive attrbibute
    C_Y0.append(calibration_table[0].loc["Fraction where Y = 1"])
    C_Yhat0.append(calibration_table[0].loc["Fraction where Y_ = 1"])
    C_Ystar0.append(calibration_table[0].loc["Fraction where Y* = 1"])
    
    # 1 is the sensitive attribute
    C_Y1.append(calibration_table[1].loc["Fraction where Y = 1"])
    C_Yhat1.append(calibration_table[1].loc["Fraction where Y_ = 1"])
    C_Ystar1.append(calibration_table[1].loc["Fraction where Y* = 1"])

In [ ]:
x = np.array([0, 1.5, 3, 4.5])
width = 0.4
label_font = {
    'color':  'k',
    'weight': 'heavy',
    'size': 10
}

title_font = {
    'color':  'k',
    'size': 14
}

attributes =[categories[i] + "=\n" + targets[i] for i in range(len(categories))]
plt.figure(figsize=(7,5), dpi=100)
plt.title("Impact of Fairness Adjustment on Calibration (Sensitive Group)", fontdict=title_font)
plt.bar(x - width,  C_Y1, width = width, color="dodgerblue")
plt.bar(x, C_Yhat1, width = width, color="coral")
plt.bar(x + width, C_Ystar1 ,width=width, color="mediumseagreen")

plt.xticks(x, attributes)
plt.xlabel("Sensitive Group Adjusted For", fontdict=label_font)
plt.ylabel("Fraction of Positive Label", fontdict=label_font)
plt.legend(["True Labels ($Y$)", "Original Model ($\hat{Y}$)", "Adjusted Model ($Y^{*}$)"], loc="upper left")
plt.savefig("Figures/calibration-sensitive.jpg")

In [ ]:
x = np.array([0, 1.5, 3, 4.5])
width = 0.4
label_font = {
    'color':  'k',
    'weight': 'heavy',
    'size': 10
}

title_font = {
    'color':  'k',
    'size': 14
}

attributes =[categories[i] + "!=\n" + targets[i] for i in range(len(categories))]
plt.figure(figsize=(7,5), dpi=100)
plt.title("Impact of Fairness Adjustment on Calibration (Non-sensitive Group)", fontdict=title_font)
plt.bar(x - width,  C_Y0, width = width, color="dodgerblue")
plt.bar(x, C_Yhat0, width = width, color="coral")
plt.bar(x + width, C_Ystar0 ,width=width, color="mediumseagreen")

plt.xticks(x, attributes)
plt.xlabel("Sensitive Group Adjusted For", fontdict=label_font)
plt.ylabel("Fraction of Positive Label", fontdict=label_font)
plt.legend(["True Labels ($Y$)", "Original Model ($\hat{Y}$)", "Adjusted Model ($Y^{*}$)"], loc="lower right")
plt.savefig("Figures/calibration-nonsensitive.jpg")

### Calibration vs Model Complexity

In [ ]:
C_Y_MC = []
C_Yhat_MC = []
C_Ystar_MC = []

Y_acc_MC = []
Y_acc_adj_MC = []

category = "sex"
sensitive_attribute = "Female"

layer_sizes = [(1), (64), (128, 64, 32), (1024, 512, 256, 128, 64)]

for layers in layer_sizes:
    
    frequency = sum(df_train[category + "_ " + sensitive_attribute])/len(df_train)
    print(f"People with {category}={sensitive_attribute} account for {frequency*100:.4}% of the training data")
    a = np.array(df_test[category + "_ " + sensitive_attribute])
    
    
    new_model = MLPClassifier(hidden_layer_sizes=layers, 
                              learning_rate_init=.003, 
                              verbose=True, 
                              learning_rate="adaptive", 
                              max_iter=300)
    
    new_model = new_model.fit(df_train.loc[:, df_train.columns != "output"], df_train["output"])
    
    y_pred =  new_model.predict(df_test.loc[:, df_train.columns != "output"])
    y_true = np.array(df_test["output"])
    Y_acc_adj_MC.append(sum(y_true == y_pred)/len(y_true))
    
    # Fit balancer to adjust the predictions
    pb = BinaryBalancer(y=y_true, y_=y_pred, a=a, summary=False)
    pb.adjust(goal='odds', summary=False)

    calibration_table, _ = pb.summary(return_stats=True)
    
    Y_acc_adj_MC.append(sum(y_true == y_adj)/len(y_true))
    # 1 is the sensitive attribute
    C_Y_MC.append(calibration_table[1].loc["Fraction where Y = 1"])
    C_Yhat_MC.append(calibration_table[1].loc["Fraction where Y_ = 1"])
    C_Ystar_MC.append(calibration_table[1].loc["Fraction where Y* = 1"])

In [ ]:
x = np.array([0, 1.5, 3, 4.5])
width = 0.4
label_font = {
    'color':  'k',
    'weight': 'heavy',
    'size': 10
}

title_font = {
    'color':  'k',
    'size': 14
}

plt.figure(figsize=(7,5), dpi=100)
plt.title("Impact of Fairness Adjustment on Calibration (vs Model Complexity)", fontdict=title_font)
# plot data in grouped manner of bar type
plt.bar(x - width,  C_Y_MC, width = width, color="dodgerblue")
plt.bar(x, C_Yhat_MC, width = width, color="coral")
plt.bar(x + width, C_Ystar_MC, width=width, color="mediumseagreen")

plt.xticks(x, ["[1]", "[64]", "[128, 64, 32]", "[1024, 512, 256, \n128, 64]"])
plt.xlabel("Hidden Layer Sizes", fontdict=label_font)
plt.ylabel("Fraction of Positive Label", fontdict=label_font)
plt.legend(["True Labels ($Y$)", "Original Model ($\hat{Y}$)", "Adjusted Model ($Y^{*}$)"], loc="upper left")
plt.savefig("Figures/calibration-nonsensitive-complexity.jpg")

In [ ]:
Y_accs_on_group = []
Y_adj_accs_on_group = []
proportion_of_dataset = []
for attribute in zip(categories, targets):
    
    category = attribute[0]
    sensitive_attribute = attribute[1]
    frequency = sum(df_train[category + "_ " + sensitive_attribute])/len(df_train)
    proportion_of_dataset.append(frequency)
    print(f"People with {category}={sensitive_attribute} account for {frequency*100:.4}% of the training data")
    a = np.array(df_test[category + "_ " + sensitive_attribute])

    y_pred =  new_model.predict(df_test.loc[:, df_test.columns != "output"])
    y_true = np.array(df_test["output"])
    
    a_bool = a.astype(bool)

    group_acc = sum(y_pred[a_bool] == y_true[a_bool])/len(y_pred[a_bool])
    print(f"The model's loss on this group (0) is {1 - group_acc:.4}")
    Y_accs_on_group.append(group_acc)
    # Fit balancer to adjust the predictions
#     print("Before Adjustment\n", confusion_matrix(y_true[a_bool], y_pred[a_bool]), "\n-----")
    pb = BinaryBalancer(y=y_true, y_=y_pred, a=a, summary=False)
    pb.adjust(goal='odds', summary=False)
    y_adj = pb.get_adjusted_predictions()
    fair_group_acc = sum(y_adj[a_bool] == y_true[a_bool])/len(y_true[a_bool])
    Y_adj_accs_on_group.append(fair_group_acc)
#     print("After Adjustment\n", confusion_matrix(y_true[a_bool], y_adj[a_bool]), "\n-------")
    print(f"The model's loss on this group (0) after adjustment is {1 - fair_group_acc:.4}\n")

In [ ]:
# create data
x = np.arange(len(categories))
width = 0.4
label_font = {
    'color':  'k',
    'weight': 'heavy',
    'size': 10
}

title_font = {
    'color':  'k',
    'size': 14
}

attributes =[categories[i] + "=\n" + targets[i] for i in range(len(categories))]
plt.figure(figsize=(7,5), dpi=100)
plt.title("Impact of Fairness Adjustment on Utility (Overparameterized Model)", fontdict=title_font)
# plot data in grouped manner of bar type
plt.bar(x - width,  Y_accs_on_group, width = width, color="dodgerblue")
plt.bar(x, Y_adj_accs_on_group, width = width, color="coral")

plt.xticks(x - width/2, attributes)
plt.xlabel("Sensitive Group Adjusted For", fontdict=label_font)
plt.ylabel("Prediction Accuracy on Sensitive Group", fontdict=label_font)
plt.legend(["Original Model ($\hat{Y}$)", "Adjusted Model ($Y^{*}$)"], loc="lower right")
# plt.savefig("Figures/subgroup_accuracy_overparameterized.jpg")

In [ ]:
categories = ["native-country", "race", "sex", "workclass"]
targets = ["Mexico", "Black", "Female", "Private"]

Y_accs = []
Y_adj_accs = []
for attribute in zip(categories, targets):
    
    category = attribute[0]
    sensitive_attribute = attribute[1]
    
    
    
    frequency = sum(df_train[category + "_ " + sensitive_attribute])/len(df_train)
    print(f"People with {category}={sensitive_attribute} account for {frequency*100:.4}% of the training data")
    a = np.array(df_test[category + "_ " + sensitive_attribute])

    y_pred =  new_model.predict(df_test.loc[:, df_train.columns != "output"])
    y_true = np.array(df_test["output"])
    acc = sum(y_pred == y_true)/len(y_pred)
    print(f"The model's overall accuracy is {acc*100:.4}%")
    Y_accs.append(acc)
    
    # Fit balancer to adjust the predictions
    pb = BinaryBalancer(y=y_true, y_=y_pred, a=a, summary=False)
    pb.adjust(goal='odds', summary=False)
    fair_acc = sum(pb.get_adjusted_predictions() == y_true)/len(y_true)
    print(f"The model's overall accuracy after adjustment is {fair_acc*100:.4}%\n")
    Y_adj_accs.append(fair_acc)

    
x = np.arange(len(categories) + 1)
width = 0.4
label_font = {
    'color':  'k',
    'weight': 'heavy',
    'size': 10
}

title_font = {
    'color':  'k',
    'size': 14
}

attributes = ["No \nAdjustment"] + [categories[i] + "=\n" + targets[i] for i in range(len(categories))]
plt.figure(figsize=(7,5), dpi=100)
plt.title("Impact of Fairness Adjustment on Utility (Overparameterized Model)", fontdict=title_font)
plt.bar(x, [Y_accs[0]] + Y_adj_accs, color=["darkgrey", "dodgerblue", "coral", "khaki", "mediumseagreen"])

plt.xticks(x, attributes)
plt.xlabel("Sensitive Group Adjusted For", fontdict=label_font)
plt.ylabel("Overall Prediction Accuracy", fontdict=label_font)


plt.ylim(0,1)
ax = plt.gca()
ax.autoscale(False)
ax.plot(plt.gca().get_xlim(), [Y_accs[0], Y_accs[0]], c="r", linestyle="--")
plt.draw()
# plt.savefig("Figures/overall_accuracy_overparameterized.jpg")